In [1]:
!pip install mediapipe
!pip install cvzone

In [2]:
import cv2
import numpy
import mediapipe as mp
import cvzone
import os
import random
from PIL import Image
from cvzone.HandTrackingModule import HandDetector
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands


In [3]:
class DragImg():
    def __init__(self, path, posOrigin, size, imgType):
 
        self.posOrigin = posOrigin
        self.imgType = imgType
        self.path = path
 
        if self.imgType == 'png':
            self.img = cv2.imread(self.path, cv2.IMREAD_UNCHANGED)
        else:
            self.img = cv2.imread(self.path)
 
        self.img = cv2.resize(self.img, (0,0),None,size,size)
 
        self.size = self.img.shape[:2]
        print(self.size)
 
    def update(self, cursor):
        ox, oy = self.posOrigin
        h, w = self.size
 
        # Check if in region
        if ox < cursor[0] < ox + w and oy < cursor[1] < oy + h:
            self.posOrigin = cursor[0] - w // 2, cursor[1] - h // 2

In [4]:
path = 'images'
paths = os.listdir(path)

In [5]:
paths.remove('.DS_Store')

In [6]:
paths

['sun_bg.png', 'hills_bg.png', 'tree.png', 'sandwich.png', 'beach_bg.png']

In [7]:
images = []
backgrounds = []
for x, img_path in enumerate(paths):
    print(img_path)
    if 'png' in img_path:
        img_type = 'png'
    else:
        img_type = 'jpg'
    if 'bg' in img_path:
#         print('bg:',img_path)
        size = 0.2
        print(size,[50 + x * 100, 50 + x * 10])
#         images.append(DragImg(f'{path}/{img_path}', [50 + x * 200, 50], 0.6,img_type))
    else:
        size = 0.1
        print(size,[50 + x * 100, 50 + x * 10])
    images.append(DragImg(f'{path}/{img_path}', [50 + x * 200, 50 + x * 10], size,img_type))

sun_bg.png
0.2 [50, 50]
(94, 172)
hills_bg.png
0.2 [150, 60]
(144, 256)
tree.png
0.1 [250, 70]
(127, 130)
sandwich.png
0.1 [350, 80]
(86, 97)
beach_bg.png
0.2 [450, 90]
(140, 172)


In [8]:
print(images)

[<__main__.DragImg object at 0x7fd4b888b820>, <__main__.DragImg object at 0x7fd4b889c8b0>, <__main__.DragImg object at 0x7fd4b889c850>, <__main__.DragImg object at 0x7fd4b889cb20>, <__main__.DragImg object at 0x7fd4b889c9a0>]


In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
 
detector = HandDetector(detectionCon=0.8)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    rect1 = cv2.rectangle(img, (1200,400), (900, 700), (255, 0, 0), 2)
    rect2 = cv2.rectangle(img, (1200,50), (900, 350), (0, 255, 0), 2)
#     cv2.imshow('box1',img[50:350,900:1200])
    
    if hands:
        lmList = hands[0]['lmList']
#         print(lmList)
#         print(lmList[8][:2])
#         print(lmList[12][:2])
        # Check if clicked
        length, info, img = detector.findDistance(lmList[8][:2], lmList[12][:2], img)
#         print(length)
        if length < 60:
            cursor = lmList[8]
            for img_obj in images:
                img_obj.update(cursor)
#     try:
    foreground, background = None, None
    for imgObject in images:
        print(imgObject)
        # Draw for JPG image
        h, w = imgObject.size
        print(imgObject.size)

        ox, oy = imgObject.posOrigin
        if imgObject.imgType == "png":
            # Draw for PNG Images
            img = cvzone.overlayPNG(img, imgObject.img, [ox, oy])
        else:
            img[oy:oy + h, ox:ox + w] = imgObject.img

        if 900 <= ox <= 1200 and 400 <= oy <= 700:
#             print('ox,oy',ox,oy)
            foreground = imgObject.img
            cv2.imwrite('fg_bg/foreground.png', foreground)
#             fg_bg.remove('.DS_Store')
        if 900 <= ox <= 1200 and 50 <= oy <= 350:
#             print('ox_bg,oy_bg', ox,oy)
            background = imgObject.img
            cv2.imwrite('fg_bg/background.png', background)
#             fg_bg.remove('.DS_Store')
    if background is not None and foreground is not None:
        bg = Image.open('fg_bg/background.png')
        fg = Image.open('fg_bg/foreground.png')
        bg_width, bg_height = bg.size
        bg_width = bg_width + 20
        bg_height = bg_height + 20
        pos_x = random.randint(0,bg_width - 10)
        pos_y = random.randint(0,bg_height - 10)

        fg_resized = fg.resize((int(bg_width/5), int(bg_height/5)), Image.LANCZOS)
        bg.paste(fg_resized,(pos_x,pos_y) , fg_resized)
        bg.save("overlay.png")
        added = cv2.imread('overlay.png',cv2.IMREAD_UNCHANGED)
        added = cv2.resize(added, (500, 300))  
        cv2.imshow('Combined image', added)

#     except:
#         pass

    cv2.imshow("Image", img)
    cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[/Users/build/sandbox/BuildRepos/Sol-Windows-1/External/VirtualMonitor-macOS/MersiveRelayCam/RelayPlugin/VtcComLib/VtcComLib.cpp] initializer()
[/Users/build/sandbox/BuildRepos/Sol-Windows-1/External/VirtualMonitor-macOS/MersiveRelayCam/RelayPlugin/avCam/avCam/avCamLib.mm] initializer()
	05-29-2022  17:57:12.769 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-29-2022  17:57:12.356 LocalCamera::passResolution set resolution to 1280x720
	05-29-2022  17:57:12.929 Local Camera start, ret=0
	05-29-2022  17:57:12.325 @@@@===RelayStream:: set kCMIOStreamPropertyFormatDescription sub=2vuy w=1280 h=720
	05-29-2022  17:57:12.176 Local Camera start, ret=0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(1

(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(14

(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(1

...

(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(14

(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(

(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(1

(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(

(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
(94, 172)
(144, 256)
(127, 130)
(86, 97)
(140, 172)
